# Kiểm tra dữ liệu

## Cấu hình & import

In [1]:
import os, glob, random, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ==== CẤU HÌNH ====
DATA_ROOT = "/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo"
SPLITS = ["train", "test"]

## Hàm tiện ích

In [2]:
def list_images_labels(split):
    img_dir = f"{DATA_ROOT}/images/{split}"
    lbl_dir = f"{DATA_ROOT}/labels/{split}"
    imgs = sorted(glob.glob(f"{img_dir}/*.jpg"))
    lbls = sorted(glob.glob(f"{lbl_dir}/*.txt"))
    return imgs, lbls

def read_label_file(path):
    with open(path, "r") as f:
        lines = [line.strip().split() for line in f.readlines() if line.strip()]
    return lines

## Kiểm tra tổng quan train/val

In [3]:
for split in SPLITS:
    imgs, lbls = list_images_labels(split)
    img_stems = {os.path.splitext(os.path.basename(p))[0] for p in imgs}
    lbl_stems = {os.path.splitext(os.path.basename(p))[0] for p in lbls}

    no_label = img_stems - lbl_stems
    orphan_lbl = lbl_stems - img_stems

    print(f"[{split.upper()}]")
    print(f"Ảnh: {len(imgs)} | Nhãn: {len(lbls)}")
    print(f"- Ảnh không có nhãn: {len(no_label)}")
    print(f"- Nhãn mồ côi: {len(orphan_lbl)}")
    print("-"*40)

[TRAIN]
Ảnh: 6034 | Nhãn: 6034
- Ảnh không có nhãn: 0
- Nhãn mồ côi: 0
----------------------------------------
[TEST]
Ảnh: 3022 | Nhãn: 3022
- Ảnh không có nhãn: 0
- Nhãn mồ côi: 0
----------------------------------------


In [4]:
# import cv2
# import matplotlib.pyplot as plt

# # --- Hàm đọc và visualize label YOLO ---
# def visualize_yolo_image(img_path, label_path):
#     img = cv2.imread(img_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     h, w, _ = img.shape

#     if not os.path.exists(label_path):
#         print(f"⚠️ Không có label cho {img_path}")
#         return

#     with open(label_path, "r") as f:
#         lines = f.readlines()

#     for line in lines:
#         cls, x, y, bw, bh = map(float, line.strip().split())
#         x_center, y_center, bw, bh = x * w, y * h, bw * w, bh * h
#         xmin = int(x_center - bw / 2)
#         ymin = int(y_center - bh / 2)
#         xmax = int(x_center + bw / 2)
#         ymax = int(y_center + bh / 2)

#         cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
#         cv2.putText(img, f"{int(cls)}", (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

#     plt.imshow(img)
#     plt.axis("off")
#     plt.show()


# # --- Lấy 3 ảnh có nhãn ---
# split = "train"
# img_dir = os.path.join(DATA_ROOT, "images", split)
# lbl_dir = os.path.join(DATA_ROOT, "labels", split)

# labeled_imgs = [
#     f for f in os.listdir(lbl_dir)
#     if os.path.isfile(os.path.join(lbl_dir, f))
# ]

# print(f"🔍 Tìm thấy {len(labeled_imgs)} ảnh có nhãn trong {split}")

# for i in range(3):
#     img_id = os.path.splitext(labeled_imgs[i])[0]
#     img_path = os.path.join(img_dir, f"{img_id}.jpg")
#     label_path = os.path.join(lbl_dir, f"{img_id}.txt")

#     print(f"\n🖼️ Hiển thị: {img_id}")
#     visualize_yolo_image(img_path, label_path)

# Kiểm tra tổng quan kiến trúc của 3 model YOLOv5n, YOLOv8n, YOLO11n

In [5]:
!pip install -q ultralytics thop --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:00


In [6]:
import os
import time
import torch
import pandas as pd
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Kiến trúc YOLO5n

In [7]:
model = YOLO('yolov5n.pt', verbose=False)
model.info(detailed=True)

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False        1728       [16, 3, 6, 6] -0.000638    0.0907        float32
    1                       model.0.bn.weight         BatchNorm2d     False          16                [16]      2.46      1.33        float32
    1                         model.0.bn.bias         BatchNorm2d     False          16                [16]     0.602      3.35        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weig

(153, 2654816, 0, 7.840102399999999)

## Phân tích kiến trúc YOLOv5n:

| Thành phần              | Giá trị                                                            |
| ----------------------- | ------------------------------------------------------------------ |
| Số lớp (layers)         | **153**                                                            |
| Số tham số (parameters) | **≈ 2.65 M**                                                       |
| Dtype                   | `float32`                                                          |
| GFLOPs                   | **7.8**                                                        |

**Kiến trúc 3-giai đoạn YOLO5n**

**🧠 Backbone (Feature Extraction)**

* Các khối từ model.0 → model.8 tạo thành backbone.

* Sử dụng Conv + BatchNorm + SiLU làm đơn vị cơ bản (C1, C2f).

* Các block model.2, model.4, model.6, model.8 chứa BottleneckCSP rút gọn → giúp giảm FLOPs ~ 40% so với YOLOv5s.

* Tăng cường khả năng trích xuất đặc trưng đa tỷ lệ (1/8, 1/16, 1/32).

**🔗 Neck (Feature Fusion)**

* Các module Concat, Upsample, và C3 (ví dụ model.10 → model.20) là phần PANet neck, hợp nhất feature maps từ các tầng khác nhau.

* Mục tiêu: kết hợp thông tin không gian (tầng thấp) và ngữ nghĩa (tầng cao) → giúp mô hình phát hiện vật thể lớn + nhỏ đồng thời.

**🎯 Head (Detection)**

* Các lớp cuối model.24 trở đi gồm:

    * cv3.x.x: khối Conv3x3 riêng cho mỗi scale (80×80, 40×40, 20×20).

    * dfl.conv.weight: Distribution Focal Loss head (từ YOLOv5u/YOLOv8) → tăng độ chính xác bounding box.

**Sử dụng cơ chế anchor-free (phiên bản “u”) thay cho anchor truyền thống → giúp nhanh hơn và đơn giản hóa training.**

## Kiến trúc YOLOv8n

In [8]:
model = YOLO('yolov8n.pt', verbose=False)
model.info(detailed=True)

layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]  -0.00279     0.152        float32
    1                       model.0.bn.weight         BatchNorm2d     False          16                [16]      2.97      1.86        float32
    1                         model.0.bn.bias         BatchNorm2d     False          16                [16]     0.249      4.17        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  -0.00012     0.063        float32
    4                       model.1.bn.weight         BatchNorm2d     False          32                [32]      5.02      1.12        float32
    4         

(129, 3157200, 0, 8.8575488)

## Phân tích kiến trúc YOLOv8n

| Thành phần              | Giá trị                                                            |
| ----------------------- | ------------------------------------------------------------------ |
| Số lớp (layers)         | **129**                                                            |
| Số tham số (parameters) | **≈ 3.16 M**                                                       |
| Dtype                   | `float32`                                                          |
| GFLOPs                   | **8.9**                                                        |

**Kiến trúc 3 giai đoạn YOLOv8n**

**🧠 Backbone**

* Các lớp model.0 → model.9: chuỗi Conv + C2f block.

* Sử dụng khối C2f (Cross-Stage Partial Fusion), kế thừa từ CSPDarknet nhưng rút gọn và hợp nhất, giúp giảm FLOPs nhưng vẫn giữ thông tin gradient xuyên tầng.

* Tầng cuối backbone là SPPF (Spatial Pyramid Pooling Fast) ở model.9, dùng để gom đặc trưng ở nhiều receptive field.

**🔗 Neck (Feature Fusion)**

* Các lớp từ model.10 đến model.21 là phần neck (PAN-FPN):

* Upsample, Concat, và Conv xen kẽ để kết hợp feature maps ở 3 tỉ lệ (P3, P4, P5).

* Giúp mô hình phát hiện vật thể ở nhiều kích cỡ.

**🎯 Head (Detection)**

* Bắt đầu từ model.22, gồm các nhánh cv2, cv3, và cuối cùng là DFL (Distribution Focal Loss) head:

    * Đây là anchor-free head (giống YOLOv5u/YOLOv11).

    * dfl.conv.weight chứa 16 bins → bounding box regression chính xác hơn.

    * Mỗi scale (80×80, 40×40, 20×20) đều có nhánh phát hiện riêng.

## Kiến trúc YOLOv11n

In [9]:
model = YOLO('yolo11n.pt', verbose=False)
model.info(detailed=True)

layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]  -0.00165      0.14        float32
    1                       model.0.bn.weight         BatchNorm2d     False          16                [16]      3.07       1.9        float32
    1                         model.0.bn.bias         BatchNorm2d     False          16                [16]     0.805      4.26        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  0.000659    0.0564        float32
    4                       model.1.bn.weight         BatchNorm2d     False          32                [32]      3.77     0.892        float32
    4         

(181, 2624080, 0, 6.614336)

## Phân tích kiến trúc YOLO11n

| Thành phần              | Giá trị                                                            |
| ----------------------- | ------------------------------------------------------------------ |
| Số lớp (layers)         | **181**                                                            |
| Số tham số (parameters) | **≈ 2.62M**                                                       |
| Dtype                   | `float32`                                                          |
| GFLOPs                   | **6.6**                                                        |


**Kiến trúc 3 giai đoạn YOLOv11n**

**⚙️ Backbone – GELAN (Generalized Efficient Layer Aggregation Network)**

* Các lớp model.0 → model.10 là backbone.

* Dùng GELAN block (thay CSP/ELAN), tối ưu hóa flow thông tin và giảm độ trễ.

* Tích hợp cơ chế attention nhẹ (attn.qkv) giúp tập trung đặc trưng không gian.

* Các thông số như cv1, cv2, m.0.m.0 thể hiện cấu trúc C2f + attention fusion — vừa giữ trọng số nhẹ vừa tăng khả năng học đặc trưng phức tạp.

**🔗 Neck – FPN & PAN Aggregation**

* Các lớp model.11 đến model.21 đảm nhận việc kết hợp đặc trưng đa tỉ lệ (Upsample + Concat).

* Tương tự YOLOv8, nhưng có thêm attention fusion giúp tăng mAP50-95 khoảng +1%.

**🎯 Head – DFLv2 Detection**

* Các lớp model.22 → model.23 là detection head:

    * Dùng Distribution Focal Loss v2 (dfl.conv.weight) – head anchor-free cải tiến.

    * Có 3 nhánh phát hiện (P3, P4, P5) tương ứng 80×80, 40×40, 20×20.

# Xây dựng kiến trúc YOLO_plus với Ultralytics

## Phiên bản 1:
| Thành phần                  | Thay đổi bạn thực hiện                              | Ý nghĩa                               |
| --------------------------- | --------------------------------------------------- | ------------------------------------- |
| **nc = 1**                  | Chỉ nhận diện 1 class                               | giảm tải rất mạnh cho Detection head  |
| **reg_max = 8**             | Giảm số bin của Distribution Focal Loss (từ 16 → 8) | giảm ~50 % tính toán regression       |
| **width_multiple = 0.25**   | Giảm độ rộng kênh so với YOLOv8n mặc định (0.33)    | giảm số tham số & FLOPs               |
| **depth_multiple = 0.33**   | Giữ nguyên độ sâu chuẩn                             | tránh mất mát khả năng học            |
| **Loại bỏ anchor**          | (anchor-free head mặc định của v8)                  | tối giản head, phù hợp edge device    |
| **Concat hợp lý (P3 – P5)** | Giữ đủ 3 scale detection                            | vẫn đa tỉ lệ, không giảm mAP mạnh     |
| **SPPF 5×5**                | pooling nhanh, giảm latency                         | giữ context vùng rộng mà ít tính toán |


In [10]:
import torch
from ultralytics.nn.tasks import DetectionModel
import yaml
import os

# ==========================================================
# 🧠 YOLO_plus (Custom Slim YOLOv8n)
# ==========================================================
yolo_plus_v1 = {
    "nc": 1,             # 1 class duy nhất
    "reg_max": 8,        # giảm từ 16 → 8 để nhẹ hơn
    "depth_multiple": 0.33,
    "width_multiple": 0.25,

    # ---------- Backbone ----------
    "backbone": [
        [-1, 1, "Conv", [32, 3, 2]],          # 0-P1/2
        [-1, 1, "Conv", [64, 3, 2]],          # 1-P2/4
        [-1, 3, "C2f", [64, True]],           # 2
        [-1, 1, "Conv", [128, 3, 2]],         # 3-P3/8
        [-1, 6, "C2f", [128, True]],          # 4
        [-1, 1, "Conv", [256, 3, 2]],         # 5-P4/16
        [-1, 6, "C2f", [256, True]],          # 6
        [-1, 1, "Conv", [512, 3, 2]],         # 7-P5/32
        [-1, 3, "C2f", [512, True]],          # 8
        [-1, 1, "SPPF", [512, 5]],            # 9
    ],

    # ---------- Head (Neck + Detect) ----------
    "head": [
        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],   # 10 ↑ x2 (20→40)
        [[-1, 6], 1, "Concat", [1]],                    # 11 concat P4
        [-1, 3, "C2f", [256, True]],                    # 12

        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],   # 13 ↑ x2 (40→80)
        [[-1, 4], 1, "Concat", [1]],                    # 14 concat P3
        [-1, 3, "C2f", [128, True]],                    # 15 (P3 head)

        [-1, 1, "Conv", [256, 3, 2]],                   # 16 ↓ x2 (80→40)
        [[-1, 12], 1, "Concat", [1]],                   # 17 concat P4
        [-1, 3, "C2f", [256, True]],                    # 18 (P4 head)

        [-1, 1, "Conv", [512, 3, 2]],                   # 19 ↓ x2 (40→20)
        [[-1, 9], 1, "Concat", [1]],                    # 20 concat P5
        [-1, 3, "C2f", [512, True]],                    # 21 (P5 head)

        [[15, 18, 21], 1, "Detect", [1]],            # 22 Detect(P3, P4, P5)
    ],
}

# Đường dẫn lưu file
save_dir = "/kaggle/working/custom_yolo"
os.makedirs(save_dir, exist_ok=True)  # ✅ Tạo thư mục nếu chưa có
yaml_path = os.path.join(save_dir, "yolo_plus_v1.yaml")

# Lưu file YAML
with open(yaml_path, "w") as f:
    yaml.dump(yolo_plus_v1, f, sort_keys=False)
print(f"✅ Đã lưu file cấu hình tại: {yaml_path}")

# Khởi tạo model trực tiếp từ dict
model = DetectionModel(cfg=yolo_plus_v1, verbose=False)
model.info(detailed=True)


✅ Đã lưu file cấu hình tại: /kaggle/working/custom_yolo/yolo_plus_v1.yaml
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d      True         216        [8, 3, 3, 3]  0.000491     0.111        float32
    1                       model.0.bn.weight         BatchNorm2d      True           8                 [8]         1         0        float32
    1                         model.0.bn.bias         BatchNorm2d      True           8                 [8]         0         0        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d      True        1152       [16, 8, 3, 3]  -0.00301    0.0686        float32
    4                       model.1.bn.weight         BatchNorm2d      True        

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/torch_utils.py:332: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  f"{i:>5g}{f'{mn}.{pn}':>40}{mt:>20}{p.requires_grad!r:>10}{p.numel():>12g}{list(p.shape)!s:>20}{p.mean():>10.3g}{p.std():>10.3g}{str(p.dtype).replace('torch.', ''):>15}"


(129, 969243, 969227, 2.8991232)

## Phiên bản 2:
| Thành phần            | YOLO_plus_v1        | YOLO_plus_v2                    | Ảnh hưởng                          |
| --------------------- | ------------------- | ------------------------------- | ---------------------------------- |
| **Base channels**     | 32 → 512            | 48 → 768 (× 1.5)                | tăng chiều sâu biểu diễn đặc trưng |
| **width_multiple**    | 0.25                | 0.25 (giữ nguyên)               | giữ scale nhẹ                      |
| **Channel scaling**   | Giảm trực tiếp      | Giảm theo tỉ lệ 0.75 từ YOLOv8n | nhất quán hơn                      |
| **DFL bins**          | 8                   | 8                               | nhẹ hơn YOLOv8n (16 bins)          |
| **Số block C2f**      | Giữ 3–6             | Giữ nguyên                      | đảm bảo receptive field tốt hơn    |
| **Params**   | ≈ 0.97 M            | ≈ 1.8 M                         | nhẹ hơn ~30 %                      |
| **FLOPs**             | ≈ 2.9 G             | ≈ 5.2 G                         | giảm ~15 % tính toán               |
| **Hiệu suất kỳ vọng** | mAP ≈ YOLOv8n − 5 % | mAP ≈ YOLOv8n − 2 %           | trade-off tốt hơn                  |



In [11]:
import torch
from ultralytics.nn.tasks import DetectionModel
import yaml

# ==========================================================
# 🧠 YOLO_plus_v2 (Medium-Light Custom)
# Mục tiêu: ~1.7M Params (Trade-off cân bằng nhất)
# ==========================================================
yolo_plus_v2 = {
    "nc": 1,             # 1 class
    "reg_max": 8,        # Giữ nguyên tối ưu hóa DFL bin cho RPi
    "depth_multiple": 0.33,
    "width_multiple": 0.25,

    # ---------- Backbone (Scale 0.75 so với v8n gốc) ----------
    # Các số kênh được tính: 64*0.75=48, 128*0.75=96, ...
    "backbone": [
        [-1, 1, "Conv", [48, 3, 2]],          # 0-P1/2 (48 channels)
        [-1, 1, "Conv", [96, 3, 2]],          # 1-P2/4 (96 channels)
        [-1, 3, "C2f", [96, True]],           # 2
        [-1, 1, "Conv", [192, 3, 2]],         # 3-P3/8 (192 channels)
        [-1, 6, "C2f", [192, True]],          # 4
        [-1, 1, "Conv", [384, 3, 2]],         # 5-P4/16 (384 channels)
        [-1, 6, "C2f", [384, True]],          # 6
        [-1, 1, "Conv", [768, 3, 2]],         # 7-P5/32 (768 channels)
        [-1, 3, "C2f", [768, True]],          # 8
        [-1, 1, "SPPF", [768, 5]],            # 9
    ],

    # ---------- Head (Scale 0.75 tương ứng) ----------
    "head": [
        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
        [[-1, 6], 1, "Concat", [1]],          # concat P4
        [-1, 3, "C2f", [384, True]],          # 12 (Scale 0.75)

        [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
        [[-1, 4], 1, "Concat", [1]],          # concat P3
        [-1, 3, "C2f", [192, True]],          # 15 (P3 head - Scale 0.75)

        [-1, 1, "Conv", [192, 3, 2]],         # 16 ↓ x2
        [[-1, 12], 1, "Concat", [1]],         # concat P4
        [-1, 3, "C2f", [384, True]],          # 18 (P4 head - Scale 0.75)

        [-1, 1, "Conv", [384, 3, 2]],         # 19 ↓ x2
        [[-1, 9], 1, "Concat", [1]],          # concat P5
        [-1, 3, "C2f", [768, True]],          # 21 (P5 head - Scale 0.75)

        [[15, 18, 21], 1, "Detect", [1]],     # 22 Detect, [1] tương ứng với nc=1
    ],
}


# Đường dẫn lưu file
save_dir = "/kaggle/working/custom_yolo"
os.makedirs(save_dir, exist_ok=True)  # ✅ Tạo thư mục nếu chưa có
yaml_path_v2 = os.path.join(save_dir, "yolo_plus_v2.yaml")
# Lưu file YAML
with open(yaml_path_v2, "w") as f:
    yaml.dump(yolo_plus_v2, f, sort_keys=False)
print(f"✅ Đã lưu file cấu hình tại: {yaml_path_v2}")

# Khởi tạo model trực tiếp từ dict
model = DetectionModel(cfg=yolo_plus_v2, verbose=False)
model.info(detailed=True)

✅ Đã lưu file cấu hình tại: /kaggle/working/custom_yolo/yolo_plus_v2.yaml
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d      True         432       [16, 3, 3, 3]  -0.00378      0.11        float32
    1                       model.0.bn.weight         BatchNorm2d      True          16                [16]         1         0        float32
    1                         model.0.bn.bias         BatchNorm2d      True          16                [16]         0         0        float32
    2                             model.0.act                SiLU     False           0                  []         -         -              -
    3                     model.1.conv.weight              Conv2d      True        3456      [24, 16, 3, 3]  -0.00126    0.0484        float32
    4                       model.1.bn.weight         BatchNorm2d      True        

(129, 1799187, 1799171, 5.1750272)

# Training model

In [12]:
import os
import yaml

DATA_ROOT = "/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo"
DATA_YAML = "/kaggle/working/data.yaml"

# Lấy đường dẫn tuyệt đối của folder images/{split}
yaml_dict = {
    "path": DATA_ROOT,
    "train": os.path.join(DATA_ROOT, "images/train"),
    "val": os.path.join(DATA_ROOT, "images/test"),
    "nc": 1,
    "names": ["sign"]
}

# Ghi ra file YAML
with open(DATA_YAML, "w") as f:
    yaml.dump(yaml_dict, f, sort_keys=False)

print(f"✅ Tạo file data.yaml tại {DATA_YAML}")


✅ Tạo file data.yaml tại /kaggle/working/data.yaml


## File kiến trúc của 2 model custom

In [13]:
!ls /kaggle/working/custom_yolo 

yolo_plus_v1.yaml  yolo_plus_v2.yaml


## Training

In [14]:
# =========================================================
# Hàm train và log kết quả
# =========================================================
def train_and_log(model_name, cfg_yaml, data_yaml, save_dir,
                  epochs=30, imgsz=640, batch=16, device="cpu"):
    print(f"\n🚀 Training {model_name} from {cfg_yaml}")
    start_time = time.time()

    model = YOLO(cfg_yaml, task="detect")
    results = model.train(
        data=data_yaml,
        imgsz=imgsz,
        epochs=epochs,
        batch=batch,
        mosaic=0.7,
        scale=0.5,
        copy_paste=0.05,
        hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
        project=save_dir,
        name=model_name,
        device=device,
        workers=2,
        patience=5,
        pretrained=False,
        verbose=False,
    )

    elapsed = round((time.time() - start_time) / 60, 2)
    metrics = results.results_dict

    summary = {
        "Model": model_name,
        "mAP50": metrics.get("metrics/mAP50(B)", 0),
        "mAP50-95": metrics.get("metrics/mAP50-95(B)", 0),
        "Precision": metrics.get("metrics/precision(B)", 0),
        "Recall": metrics.get("metrics/recall(B)", 0),
        "Epochs": epochs,
        "TrainTime(min)": elapsed,
        "Weights": os.path.join(save_dir, model_name, "weights", "best.pt"),
    }

    print(f"✅ Done {model_name} | mAP50={summary['mAP50']:.4f} | "
          f"mAP5095={summary['mAP50-95']:.4f} ")

    return summary


## Tổng hợp kết quả

In [15]:

# =========================================================
# Khai báo model YAML và chạy huấn luyện
# =========================================================
CUSTOM_DIR = "/kaggle/working/custom_yolo"
DATA_YAML = "/kaggle/working/data.yaml"
RUNS_DIR = "/kaggle/working/yolo_custom_runs"
device = 0 if torch.cuda.is_available() else "cpu"

models_cfg = {
    "YOLO_plus_v1": os.path.join(CUSTOM_DIR, "yolo_plus_v1.yaml"),
    "YOLO_plus_v2": os.path.join(CUSTOM_DIR, "yolo_plus_v2.yaml"),
}

all_results = []
for name, cfg_path in models_cfg.items():
    result = train_and_log(
        model_name=name,
        cfg_yaml=cfg_path,
        data_yaml=DATA_YAML,
        save_dir=RUNS_DIR,
        epochs=30,
        imgsz=640,
        batch=16,
        device=device,
    )
    all_results.append(result)

# =========================================================
# 4️⃣ Lưu kết quả ra CSV
# =========================================================
df = pd.DataFrame(all_results)
csv_path = os.path.join(RUNS_DIR, "custom_models_summary.csv")
df.to_csv(csv_path, index=False)
print(f"\n📊 Saved summary metrics to: {csv_path}")
print(df)



🚀 Training YOLO_plus_v1 from /kaggle/working/custom_yolo/yolo_plus_v1.yaml
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/custom_yolo/yolo_plus_v1.yaml, momentum=0.937, mosaic=0.7, multi_scale=False, name=YOLO_plus_v1, nbs=64

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       3022       8274      0.875      0.734       0.82      0.544
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /kaggle/working/yolo_custom_runs/YOLO_plus_v1
✅ Done YOLO_plus_v1 | mAP50=0.8198 | mAP5095=0.5444 

🚀 Training YOLO_plus_v2 from /kaggle/working/custom_yolo/yolo_plus_v2.yaml
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hs

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       3022       8274      0.899      0.794       0.87      0.603
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /kaggle/working/yolo_custom_runs/YOLO_plus_v2
✅ Done YOLO_plus_v2 | mAP50=0.8704 | mAP5095=0.6034 

📊 Saved summary metrics to: /kaggle/working/yolo_custom_runs/custom_models_summary.csv
          Model     mAP50  mAP50-95  Precision    Recall  Epochs  \
0  YOLO_plus_v1  0.819802  0.544366   0.874683  0.733986      30   
1  YOLO_plus_v2  0.870442  0.603428   0.898853  0.793714      30   

   TrainTime(min)                                            Weights  
0           85.52  /kaggle/working/yolo_custom_runs/YOLO_plus_v1/...  
1           84.19  /kaggle/working/yolo_custom_runs/YOLO_plus_v2/...  
